In [ ]:
from coldquanta.qiskit_tools.modeling.neutral_atom_noise_model import create_noise_model
from coldquanta.qiskit_tools.coldquanta_sim_backend import ColdQuantaSimBackend

from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram
import math

In [ ]:
# Instead of using the built in transpiler, explicitly define
def na_h(c, q):
    # H on q1
    c.r(q=q, phi=math.pi / 2, theta=math.pi / 2)
    c.r(q=q, phi=0.0, theta=math.pi)

def na_cnot(c, q0, q1):
    # H on q1
    na_h(c, q1)
    # CZ
    c.cz(q0, q1)
    # H on q1
    na_h(c, q1)

#create a circuit
bell = QuantumCircuit(4)

na_h(bell, 0)
for i in range(3):
    na_cnot(bell, i, i+1)

bell.measure_all()

In [ ]:
neutral_atom = ColdQuantaSimBackend(rows=2, cols=2)
neutral_atom_config = neutral_atom.configuration()
coupling_map = neutral_atom_config.coupling_map

simulator = Aer.get_backend('qasm_simulator')

result_noise_free = execute(bell,
                       simulator,
                       coupling_map=coupling_map,
                       optimization_level=0).result()

counts_noise_free = result_noise_free.get_counts()

plot_histogram(counts_noise_free, title="Counts for 4-qubit GHZ state with no noise")

In [ ]:
# simulate with neutral atom noise model

# use defaults
noise_model = create_noise_model()


result_noise = execute(bell,
                       simulator,
                       noise_model=noise_model,
                       coupling_map=coupling_map,
                       optimization_level=0,
                       shots=1024).result()

counts_noise = result_noise.get_counts()

plot_histogram(counts_noise, title="Counts for 4-qubit GHZ state with noise")


In [ ]:
# Adjust noise characteristics

# decrease cz fidelity from the 0.95 default to 0.7
noise_model = create_noise_model(cz_fidelity=0.7)


result_noise = execute(bell,
                       simulator,
                       noise_model=noise_model,
                       coupling_map=coupling_map,
                       optimization_level=0,
                       shots=1024).result()

counts_noise = result_noise.get_counts()

# Note how noise states increase in count
plot_histogram(counts_noise, title="Counts for 4-qubit GHZ state with noise")

